# Imports

In [1]:
from pathlib import Path
persist_path = Path('.') / 'persist'
import sys
project_root = str(Path(sys.path[0]).parents[0].absolute())
if project_root not in sys.path:
    sys.path.append(project_root)
import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:100%; }</style>"))
from bokeh.io import output_notebook, show
from bokeh.colors import RGB
output_notebook()
from functools import partial
import scripts.utils as utils
from importlib import reload
from collections import namedtuple

Loading BokehJS ...

# Acquisition des données

In [2]:
orders = pd.read_pickle(persist_path / 'orders.pkl')
order_count = len(orders)
df_clt = pd.read_pickle(persist_path / 'clt.pkl')
clt_fields = ['seg1', 'seg2', 'seg3', 'seg4', 'cat', 'sscat', 'hier4', 'groupecompte', 'relationtype', 'pilcom']
for field in clt_fields:
    try:
        orders = orders.drop(field)
    except: 
        pass
orders = orders.merge(df_clt.loc[:, clt_fields], left_on=['orgacom', 'client'], right_index=True, validate='m:1')
if len(orders) != order_count:
    raise RuntimeError('Something went wrong when merging orders with client master data!')
    
full_orders = pd.read_pickle(persist_path / 'full_orders_with_zscore.pkl')
day_orders = pd.read_pickle(persist_path / 'day_order.pkl')

lib_seg = pd.read_pickle(persist_path / 'lib_seg.pkl')

# Bar plots

In [3]:
show(partial(utils.bk_histo_seg,
             source_df=orders.reset_index(),
             segs=['seg1', 'seg2'],
             filters={},
             filters_exclude={'groupecompte': ['ZPER']}, # 'hier4': ['198863']
             plot_kwargs=dict(title="Chiffre d'affaire brut 2019 en fonction de la segmentation niveau 2 et du canal"),
            ),
     notebook_url="http://devdm:8888")

In [4]:
show(partial(utils.bk_histo_seg,
             source_df=orders.reset_index(),
             segs=['seg2', 'seg3'],
             filters={'seg1': ['Z3']},
             filters_exclude={'groupecompte': ['ZPER']}, # 'hier4': ['198863']
             plot_kwargs=dict(title="Chiffre d'affaire brut 2019 en fonction de la segmentation niveau 3 et du canal"),
            ),
     notebook_url="http://devdm:8888")

# Bubbles

In [5]:
filter_rhd = (full_orders.seg1 == 'Z3')
filter_main_origins = ~(
    (full_orders.origin2.isin(['EDI']) & full_orders.seg3.isin(['ZI', 'ZK'])) |
    (full_orders.origin2.isin(['VR']) & full_orders.seg3.isin(['ZJ', 'ZL'])) |
    (full_orders.origin2.isin(['WEB']) & full_orders.seg3.isin(['ZJ']))
)
# filters = filter_rhd & filter_main_origins & filter_ppf
begin_date, end_date = pd.to_datetime('2017-01-01'), pd.to_datetime('2022-12-31')
show(partial(utils.bk_bubbles,
             data=full_orders,
             filters=filter_rhd & filter_main_origins,
#              compute_zscores=True,
            ),
     notebook_url="http://devdm:8888")

# Detail

### Ajout des libellés des segments et hiérarchies au dataset client

In [6]:
segs = ['seg1', 'seg2', 'seg3', 'seg4', 'cat', 'sscat']
for i, seg in enumerate(segs):
    df_clt = (
        df_clt
        .merge(lib_seg.loc[i + 1, 'designation'].rename(seg + '_lib'), left_on=seg, right_index=True, validate='m:1', how='left')
    )
names = df_clt.reset_index().loc[:, ['orgacom', 'client', 'nom']]
names = names.loc[names.client.str.isnumeric()]
names = names.astype({'client': 'int'}).astype({'client': 'str'})
hiers = ['hier4', 'hier3', 'hier2', 'hier1']
df_clt = df_clt.reset_index()
for hier in hiers:
    df_clt = (
        df_clt.merge(
            names.rename({'nom': hier + '_lib', 'client': 'client_' + hier}, axis=1),
            left_on=['orgacom', hier],
            right_on=['orgacom', 'client_' + hier],
            validate='m:1',
            how='left'
        )
    )
    df_clt = df_clt.drop('client_' + hier, axis=1)
df_clt = df_clt.set_index(['orgacom', 'client'])

### Remise dans l'ordre de l'index du dataset des commandes

In [7]:
orders = orders.swaplevel().sort_index()

### Paramètres de la visualisation

In [8]:
labeled_bins = namedtuple('labeled_bins', ['labels', 'bin_limits'])
bins = labeled_bins(
    labels=['no_web', 'web'],
    bin_limits=[0., .5, 1.001],
)
bin_colors={
    'no_web': RGB(0, 255, 0, .0),
    'web': RGB(0, 255, 0, .1),
    'full': RGB(0, 255, 0, .2),
    'inactive': RGB(100, 100, 100, .2),
}

### Visualisation du détail par cient

In [9]:
# Exemple de client
oc, clt = (
('1ALO', '0000026443')  # AESTERA : MDR LES JARDINS DE CUVIERES ==> bascule en 2019 INTERESSANT A REGARDER!
)

In [10]:
end_date = pd.to_datetime('2021-02-28')
show(
    partial(
        utils.bk_detail,
        in_data=day_orders.loc[oc, clt, :end_date],
        order_data=orders.loc[oc, clt, :end_date],
        client=clt,
        oc=oc,
        bins=bins,
        bin_colors=bin_colors,
        groupers=['orgacom', 'client'],
        inactive_duration=20,
        indicator_status='brutrevenue',
        origin='WEB',
        indicator_perf='margin',
        inactive_roll_mode='stitch',
        clt_data=df_clt.loc[oc, clt],
    ),
    notebook_url="http://devdm:8888")

### Autres clients

In [11]:
oc, clt = (
#     perim.sample(1).iloc[0]
# '1ALO', '0000170669'
# '1ALO', '0000020697'
# '1ALO', '0000020575'
# '1ALO', '0000020531'
# '1ALO', '0000024795'

# ('1ALO', '0000162867')
# ('1ALO', '0000026355')
# ('2BRE', '0000162960')
# ('2BRE', '0000135319')
# ('1SOU', '0000109546')
# ('1LRO', '0000211945')
('1SOU', '0000129993')   # TUTTI PIZZA JEAN RIEU
# ('1SOU', '0000200101') #   TUTTI PIZZA ST ORENS ==> uniquement du WEB, viison de la diminution de la fréquence
# ('1ALO', '0000196808')
# ('1SOU', '0000253271')  #  TUTTI PIZZA MONTREJEAU ==> un peu comme ST ORENS
# ('1SOU', '0000069161')  #  TUTTI PIZZA MONTAUBAN ==> véritable bascule
# ('1ALO', '0000026272')  #  CACIC : MDR SAINT CHARLES ==> Bascule
#     ('1ALO', '0000026286')  # CACIC : EHPAD MDR  SAINTE FAMILLE ==> Bascule après inactivité
#     ('2BRE', '0000247694')  # CACIC : EHPAD RESID LES BLES D'ORS ==> Full Web aevc une vente route
#     ('1LRO', '0000255792')  # CACIC : SOLANID MONTPELLIER ST ROCH ==> s'y est mis, puis a abandonné
#  ('1SOU', '0000083353')  # CACIC : LES FLORALIES ==> Quasi zéro web   
#     ('2BRE', '0000153386')  # CACIC : ADIMC 35 FOYER DES GLYCINES ==> bascule en 2018
# ('1ALO', '0000026443')  # AESTERA : MDR LES JARDINS DE CUVIERES ==> bascule en 2019 INTERESSANT A REGARDER!
# ('1ALO', '0000157265')  # AESTERA : IME DE DANNEMARIE ==> 100% WEB
#     ('1ALO', '0000206981')  # AESTERA : REST LEGUMEZ MOI ==> Celui là montre qu'on n'y a pas perdu à basculer sur le WEB
#     ('1ALO', '0000025685')  # AESTERA : LE HOME FLEURI ==> Expérimentations puis bascule
# ('1SOU', '0000243538') # le self de la mairie => que du WEB
#     ('1ALO', '0000027991')  # CORA : CAFET CORA COUDEKERQUE - DUM ==> jolie transition, s'est arrêté en 2019
#     ('1ALO', '0000027977')  # CORA : CAFET CORA VERDUN - VDM ==> idem Coudekerque
#     ('1ALO', '0000027973')  # CORA : CAFET CORA CORMONTREUIL - RCM ==> On voit en plus la progression de marge. Pareil, arrêt en 2019
# ('1ALO', '0000027971')  # CORA : CAFET CORA VESOUL - VEM ==> bascule également, fin 2019.
)

2021-01-07 15:17:31.331107: Computing totals
2021-01-07 15:17:31.339131: Done! Elapsed: 0:00:00.008034
2021-01-07 15:17:31.339232: Computing inactive periods
2021-01-07 15:17:31.345050: Done! Elapsed: 0:00:00.005826
2021-01-07 15:17:31.345105: Computing rolling indicators
2021-01-07 15:17:31.373961: Done! Elapsed: 0:00:00.028867
2021-01-07 15:17:31.374035: Computing percentage and statuses
2021-01-07 15:17:31.380088: Done! Elapsed: 0:00:00.006062
